In [1]:
import chess
import numpy as np
import torch

1. Reprezentacja stanu

In [2]:
piece_to_index = {
    chess.PAWN: 0,
    chess.KNIGHT: 1,
    chess.BISHOP: 2,
    chess.ROOK: 3,
    chess.QUEEN: 4,
    chess.KING: 5
}
def get_attacked_squares(board, color):
    """
    Zwraca SquareSet kontrolowanych pól
    """
    attacked = chess.SquareSet()
    for square in board.pieces(chess.PAWN, color):
        attacked |= board.attacks(square)
    for square in board.pieces(chess.KNIGHT, color):
        attacked |= board.attacks(square)
    for square in board.pieces(chess.BISHOP, color):
        attacked |= board.attacks(square)
    for square in board.pieces(chess.ROOK, color):
        attacked |= board.attacks(square)
    for square in board.pieces(chess.QUEEN, color):
        attacked |= board.attacks(square)
    for square in board.pieces(chess.KING, color):
        attacked |= board.attacks(square)
    return attacked

def generate_bitboards(board):
    """
    Generuje bitboardy do upakowania w tensor, każdy bitboard reprezentuje różne typy figur.
    """
    bitboards = np.zeros((7, 8, 8), dtype=np.float32)  # 6 na figury, 1 na kontrolowane pola

    # Figury
    for piece_type in piece_to_index.keys():
        index = piece_to_index[piece_type]
        
        # Białe figury
        white_squares = board.pieces(piece_type, chess.WHITE)
        for square in white_squares:
            row, col = divmod(square, 8)
            bitboards[index, row, col] = 1.0  # Białe jako +1

        # Czarne figury
        black_squares = board.pieces(piece_type, chess.BLACK)
        for square in black_squares:
            row, col = divmod(square, 8)
            bitboards[index, row, col] = -1.0  # Czarne jako -1

    # Kontrolowane pola
    controlled_squares = np.zeros((8, 8), dtype=np.float32)

    white_attacks = get_attacked_squares(board, chess.WHITE)
    for square in white_attacks:
        row, col = divmod(square, 8)
        controlled_squares[row, col] += 1.0

    black_attacks = get_attacked_squares(board, chess.BLACK)
    for square in black_attacks:
        row, col = divmod(square, 8)
        controlled_squares[row, col] -= 1.0

    # Dodaj do bitboardów kontrolowane pola
    bitboards[6] = controlled_squares
    return bitboards

# Pakuj bitboardy w tensor
def board_to_tensor(board):
    bitboards = generate_bitboards(board)
    tensor = torch.from_numpy(bitboards)
    return tensor



In [3]:

# Nowa plansza
board = chess.Board()

# Stan planszy jako tensor
state_tensor = board_to_tensor(board)

# Rozmiar tensora
print("Kształt tensora stanów: ", state_tensor.shape)  # [7, 8, 8] (na razie)
state_tensor

Kształt tensora stanów:  torch.Size([7, 8, 8])


tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [-1., -1., -1., -1., -1., -1., -1., -1.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  0.,  0., -1.,  0.]],

        [[ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0

2. Przestrzeń możliwych działań

In [4]:
def get_action_mask(board):
    action_mask = torch.zeros((8, 8, 8, 8), dtype=torch.float32)
    
    for move in board.legal_moves:
        from_square = move.from_square
        to_square = move.to_square
        
        from_row, from_col = divmod(from_square, 8)
        to_row, to_col = divmod(to_square, 8)
        
        action_mask[from_row, from_col, to_row, to_col] = 1.0

    return action_mask

action_mask = get_action_mask(board)
print("Kształt maski: ", action_mask.shape)
action_mask

Kształt maski:  torch.Size([8, 8, 8, 8])


tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 

3. Architektura sieci neuronowej

In [6]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as nnF


class MaskLayer(nn.Module):
    def __init__(self):
        super(MaskLayer, self).__init__()
    
    # Mask is made of 0s/1s, so it will set to 0 any invalid move
    def forward(self, x, mask):
        return torch.mul(x, mask)

class LuigiCNN(nn.Module):
    def __init__(self):
        super(LuigiCNN, self).__init__()

        # Convolutional layers with batch normalization
        self.conv1 = nn.Conv2d(in_channels=7, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 8 * 8 * 8 * 8)  # Output size: 4096

        # Mask layer
        self.mask_layer = MaskLayer()

    def forward(self, x, action_mask):
        # x: [batch_size, 7, 8, 8]
        batch_size = x.size(0)

        # Convolutional layers with batch normalization and ReLU
        x = nnF.relu(self.bn1(self.conv1(x)))  # [batch_size, 64, 8, 8]
        x = nnF.relu(self.bn2(self.conv2(x)))  # [batch_size, 128, 8, 8]
        x = nnF.relu(self.bn3(self.conv3(x)))  # [batch_size, 128, 8, 8]

        # Flatten
        x = x.view(batch_size, -1)  # [batch_size, 128 * 8 * 8]

        # Fully connected layers with ReLU
        x = nnF.relu(self.fc1(x))     # [batch_size, 1024]
        x = self.fc2(x)             # [batch_size, 4096]

        # Reshape to match action space
        x = x.view(batch_size, 8, 8, 8, 8)  # [batch_size, 8, 8, 8, 8]

        # Apply the mask layer
        x = self.mask_layer(x, action_mask)

        return x  # Logits
